In [1]:
from utils import *
from variables import *
from utils import parse_excel, read_pickle
from glob import glob

In [2]:
#def generate_pdfstest(auswahl, Summery_df, HedgeBedarf_df, HedgeBedarf1_df, stock_price, heute, nbd_dict, tage_bis_verfall, delta, expiry, expiry_1, file_path):

    # Fifth stage
    # Values to create arrow
    idx_closest = (HedgeBedarf_df.Basis - stock_price).abs().idxmin()
    closest_Basis =  HedgeBedarf_df.loc[idx_closest,"Basis"]
    hedge_sum = HedgeBedarf_df.HedgeSum + HedgeBedarf1_df.HedgeSum
    closest_Sum = hedge_sum.loc[idx_closest]

    x_axis_length = hedge_sum.max() - hedge_sum.min()
    y_axis_length = HedgeBedarf_df.Basis.max() - HedgeBedarf_df.Basis.min()

    min_Kontrakte = 5000
    max_Differenz = 1000
    prozentual = 0.2

    dict_verfall_sufix= {
        True : "_verfall",
        False : ""
    }

    is_close_verfall =  tage_bis_verfall < 5
    for is_detailed in [False, True]:
        if is_detailed:
            indexes = Summery_df.loc[(Summery_df.Basis >= stock_price)].tail(10).index.to_list() + Summery_df.loc[(Summery_df.Basis < stock_price)].head(10).index.to_list()
            Summery_df =  Summery_df.loc[indexes].reset_index(drop=True)

            indexes = HedgeBedarf_df.loc[(HedgeBedarf_df.Basis >= Summery_df.Basis.min()) & (HedgeBedarf_df.Basis <= Summery_df.Basis.max())].index
            
            HedgeBedarf_df = HedgeBedarf_df.loc[indexes].reset_index(drop=True)
            HedgeBedarf1_df = HedgeBedarf1_df.loc[indexes].reset_index(drop=True)

        # (CF > min_Kontrakte) AND (PF > min_Kontrakte) AND (ABS(CF - PF)  < MIN(PF,CF)*)
        mask_highlights = (
            (Summery_df.openInterest_CF > min_Kontrakte) & 
            (Summery_df.openInterest_PF > min_Kontrakte) & 
            (
                (Summery_df.openInterest_CF - Summery_df.openInterest_PF).abs() < 
                (Summery_df[['openInterest_PF',"openInterest_CF"]].min(axis=1)*prozentual)))


        ################################## Hilights for Basis column ####################################

        Summery_df["basis_color"] = "lavender"
        if not is_detailed:
            Summery_df.loc[
                mask_highlights, "basis_color"] = "yellow"
        col_basis_color = Summery_df.basis_color.to_numpy()
        #################################################################################################

        ######################### Gradient of red and blue for Anderung column ##########################
        aux = Summery_df.Änderung.reset_index()
        aux['r'] = aux['g']  = aux['b'] = 255
        aux.loc[aux.Änderung<0,'g'] = aux.loc[aux.Änderung<0,'b'] = 255 - 150*aux.Änderung/(aux.Änderung.min()) 
        aux.loc[aux.Änderung>0,'g'] = aux.loc[aux.Änderung>0,'r'] = 255 - 150*aux.Änderung/(aux.Änderung.max())
        aux = aux.astype(float)
        rb_shades = np.array([f"rgb({aux.loc[i,'r']},{aux.loc[i,'g']},{aux.loc[i,'b']})" for i in range(aux.shape[0]) ])
        col_anderung_color = rb_shades
        #################################################################################################

        ############################# Colors for heute and last_day columns #############################
        col_heute_color = posneg_binary_color(Summery_df.heute,"rgb(0, 204, 204)","rgb(77, 166, 255)")
        col_last_day_color = posneg_binary_color(Summery_df.last_day,"rgb(0, 204, 204)","rgb(77, 166, 255)")
        #################################################################################################

        ###################### Gradient of green and blue for Put anc Call columns ######################
        col_put_color  = np.array(n_colors('rgb(214, 245, 214)', 'rgb(40, 164, 40)',
            20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_PF,19)]

        col_call_color  = np.array(n_colors('rgb(204, 224, 255)', 'rgb(0, 90, 179)',
            20, colortype='rgb'))[scale_col_range(Summery_df.openInterest_CF,19)]
        #################################################################################################

        num_rows = Summery_df.shape[0]+1
        height = 1050 #row_height*42
        row_height = int(height/(num_rows))

        # width of the image
        widht = 1000
        row_height_percent =  (row_height/height)


        values_body = [
                    (Summery_df.Basis +1e-6).round(0).astype(int),
                    (Summery_df.Änderung +1e-6).round(0).astype(int),
                    (Summery_df.heute +1e-6).round(0).astype(int),
                    (Summery_df.last_day +1e-6).round(0).astype(int),
        ]

        values_header = bold(["Basis","Änderung",nbd_dict['heute'].strftime("%d/%m/%y"),nbd_dict['last'].strftime("%d/%m/%y")])


        if not  is_detailed:
            values_body += [(Summery_df.openInterest_PF), (Summery_df.openInterest_CF)]
            values_header += bold(["Put","Call"])


        font_size =  int(440/num_rows)

        fig = go.Figure(
            data=[
                go.Table(

                    # Define some paremeters for the header
                    header=dict(
                        # Names of the columns
                        values= values_header,
                        
                        # Header style
                        fill_color='paleturquoise',
                            align='center',
                            font = {'size': int(font_size*0.8)},
                            height = row_height,
                    ),

                    cells=dict(

                        align='center',
                        height = row_height,
                        font = {'size': font_size,},

                        # Values of the table
                        values= values_body,

                        # Colors of the columns
                        fill_color = [
                            col_basis_color,
                            col_anderung_color,
                            col_heute_color,
                            col_last_day_color,
                            col_put_color, 
                            col_call_color
                        ],
                    )
                )
            ],
        )
        fig.update_layout(
            height=1050, width=550,
            margin=dict(l=0,r=0,b=0.0,t=0)
        )

        fig.write_image(os.path.join(current_results_path,f'image_table{dict_verfall_sufix[is_detailed]}.svg'),scale=1)


        data =  [

            # Create 0  y axis
            go.Scatter(
                x = [0,0], 
                y = [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
                mode = "lines",
                marker_color = "orange",
                showlegend = False
            ),

            go.Scatter(
                x = hedge_sum,
                y = HedgeBedarf_df.Basis,
                mode = "lines",
                name =expiry.strftime("%Y-%m") + " + "+ expiry_1.strftime("%Y-%m"),
                marker_color= "blue"
            ),

        go.Scatter(
            x = [closest_Sum + x_axis_length/5,closest_Sum + x_axis_length/50], 
            y = [closest_Basis,closest_Basis],
            marker= dict(size=20,symbol= "arrow-bar-up", angleref="previous"),
            marker_color = "red",
            showlegend = False
        )
        ]
        
        min_x_list = [hedge_sum.min()]
        max_x_list = [hedge_sum.max()]
        if not is_detailed:
            min_x_list.append(HedgeBedarf1_df.HedgeSum.min())
            max_x_list.append(HedgeBedarf1_df.HedgeSum.max())
            data += [
                go.Scatter(
                    x = HedgeBedarf1_df.HedgeSum,
                    y = HedgeBedarf_df.Basis,
                    mode = "lines",
                    name = expiry_1.strftime("%Y-%m"),
                    marker_color= "rgb(255,0,255)"
                ),
            ]

        ##################################################################################################
        dx =  0.1*(hedge_sum.max() - hedge_sum.min())



        fig = go.Figure(data=data)

        #mask = HedgeBedarf_df.index % round(HedgeBedarf_df.shape[0]/40) == 0
        fig.update_layout(
            margin=dict(l=0,r=0,b=0.1,t=row_height),
            # Set limits in the x and y axis
            yaxis_range= [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
            xaxis_range= [min(min_x_list) - dx, max(max_x_list) + dx],
            yaxis = dict(
                tickmode = 'array',
                tickvals = Summery_df.Basis, #HedgeBedarf_df.Basis[mask],
                ticktext = Summery_df.Basis,#HedgeBedarf_df.Basis[mask]
            ),
            
            # Remove margins
            paper_bgcolor="white",
            
            # Define width and height of the image
            width=380,height=1050,
            template = "seaborn",

            # Legend parameters
            legend=dict(
                yanchor="top",
                y=1,# - (row_height/height),
                xanchor="right",
                x= 1,
                font=dict(
                    size = 8
                ),
                # legend in the vertical
                orientation = "v",
                bgcolor  = 'rgba(0,0,0,0)'
                ),
        )


        fig.write_image(os.path.join(current_results_path,f'image_graph{dict_verfall_sufix[is_detailed]}.svg'),scale=1)



        fig = go.Figure()

        if not is_detailed:
            fig.add_trace(
                trace = go.Bar(name='Put', y=Summery_df.Basis, x=-Summery_df.openInterest_PF,orientation='h', marker_color = 'rgb(40, 164, 40)'),
            )

            fig.add_trace(
                trace = go.Bar(name='Call', y=Summery_df.Basis, x=Summery_df.openInterest_CF,orientation='h', marker_color = 'rgb(0, 90, 179)'),
            )



        fig.update_layout(
            #barmode='stack', 
            barmode='overlay', 
            margin=dict(l=0,r=0,b=0.1,t=row_height),
            # Set limits in the x and y axis
            #yaxis_range= [HedgeBedarf_df.Basis.min(), HedgeBedarf_df.Basis.max()],
            #xaxis_range= [hedge_sum.min() - dx, hedge_sum.max() + dx],
            
            # Define width and height of the image
            width=80,height=1050,
            template = "seaborn",
            showlegend=False,
            bargap =0.5 ,
            plot_bgcolor='rgba(0, 0, 0, 0)',
            paper_bgcolor='rgba(0, 0, 0, 0)',
        )


        fig.update_xaxes(visible=False)
        fig.update_yaxes(visible=False)

        fig.write_image(os.path.join(current_results_path,f'image_bar{dict_verfall_sufix[is_detailed]}.svg'),scale=1)


        with open(src_html) as file:
            template = file.read()

        if not is_close_verfall:
            subst = ""
            regex = r"\$BEGIN_DETAIL\$.*\$END_DETAIL\$"
            template = re.sub(regex, subst, template, 0, re.MULTILINE | re.DOTALL)
        else:
            template = template.replace("$BEGIN_DETAIL$", "").replace("$END_DETAIL$", "")

        dict_title = {
            0 : "OpenInterest und HedgeBedarf",
            1 : "STOXX 50 OpenInterest und HedgeBedarf",
        }

        # Replace specific characters in the template by values
        dict_raplace = {
            "$PUT_SUM$": int(Summery_df.openInterest_PF.sum()),
            "$CALL_SUM$": int(Summery_df.openInterest_CF.sum()),
            "$TBF$": tage_bis_verfall,
            "$DELTA$":str(delta).replace(".",","),
            "$DATE$": heute.strftime("%d/%m/%Y"),
            "$FRONT_DATE$" : expiry.strftime("%Y-%m"),
            "$TITLE$" : dict_title[auswahl],
            "$HEADER_COLOR$" : "background-color: rgb(12, 89, 177)"
        }

        for key, value in dict_raplace.items():
            template = template.replace(key, str(value))


        result_html = {True:summery_html, False: summery_verfall_html}[is_detailed]

        # Export html file
        with open(result_html,'w') as file:
            file.write(template)

        # Results files
        
    dict_auswahl_colors = {
        0 : {"$HEADER_COLOR$":"", "$FUTURE_COLOR$":"background-color: rgb(0, 174, 255);","$FONT_COLOR$":"color: black;"},
        1 : {"$HEADER_COLOR$":"background-color: rgb(12, 89, 177);", "$FUTURE_COLOR$":"", "$FONT_COLOR$" : "color: white;"},
    }

    with open(src_css) as file:
        css_file = file.read()

    for key, value in dict_auswahl_colors[auswahl].items():
        css_file = css_file.replace(key, str(value))

    with open(result_css,'w') as file:
        file.write(css_file)

    converter.convert("file://" + os.path.join(os.getcwd(),result_html), file_path)
    #shutil.copyfile(list_pdf_files[0], list_pdf_files[1])
    
    print("PDF has been generated.")

IndentationError: unexpected indent (4046201671.py, line 5)